In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.3/359.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 32.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 104.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 15.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━

In [2]:
!pip install xgboost networkx scikit-learn --quiet

In [3]:
import json
import numpy as np
import pandas as pd
import joblib

from unsloth import FastLanguageModel
import torch
import warnings
warnings.filterwarnings("ignore")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-12-04 05:09:45.427267: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764824985.622760      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764824985.674501      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
# ==== CONFIG ====
DATA_PATH = "/kaggle/input/new-observational-data/NewObservationalDataset.csv"
SCHEMA_PATH = "/kaggle/input/variable-schema-fixed-clean/variable_schema_fixed_clean.json"              # from your previous step
SCM_BUNDLE_PATH = "/kaggle/input/sgm-xgboost-gpu/scm_xgboost_gpu.pkl"           # adjust if saved elsewhere
MODEL_ID = "unsloth/Mistral-7B-Instruct-v0.3-bnb-4bit"      # change if you use a different Unsloth model

# 4 test transcripts you chose (fill in your real IDs here)
TEST_TRANSCRIPT_IDS = [
    "1c99b47c-4e80-49d5-b3c6-52b0c3230fa8",
]

# Your test query (edit this)
USER_QUERY = "How would the interaction have changed if the agent had apologized at the moment the customer first expressed frustration?"

In [5]:
# ==== LOAD UNSLOTH MISTRAL ====
print("Loading Unsloth Mistral...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_ID,
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True,
    device_map = "balanced",
)
FastLanguageModel.for_inference(model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Loading Unsloth Mistral...
==((====))==  Unsloth 2025.11.6: Fast Mistral patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device: cuda


In [6]:
# ==== LOAD VARIABLE SCHEMA ====
with open(SCHEMA_PATH, "r", encoding="utf-8") as f:
    schema = json.load(f)

# Map var name -> schema entry for easy lookup
schema_by_name = {v["name"]: v for v in schema}

# List of numeric variable names (from schema)
NUMERIC_VARS = [v["name"] for v in schema]


# ==== LOAD DATASET ====
df_full = pd.read_csv(DATA_PATH)
print("Full dataset shape:", df_full.shape)

# Ensure all numeric vars exist
missing = [c for c in NUMERIC_VARS if c not in df_full.columns]
if missing:
    print("WARNING: These schema vars not in dataset:", missing)

# Numeric-only view used for SCM (same columns as used in training)
df_numeric = df_full[NUMERIC_VARS].copy()
print("Numeric df shape:", df_numeric.shape)


# ==== LOAD SCM BUNDLE (XGBoost SCM) ====
bundle = joblib.load(SCM_BUNDLE_PATH)
scm_models = bundle["scm_models"]
topo_order = bundle["topo_order"]

# Simulation function (if you don't already have it in your notebook)
def simulate_scm(base_row, interventions, scm_bundle):
    """
    base_row: pd.Series with numeric vars only.
    interventions: dict { var_name: new_value }
    scm_bundle: dict with "scm_models", "topo_order"
    """
    scm_models = scm_bundle["scm_models"]
    topo_order = scm_bundle["topo_order"]

    x = base_row.copy()

    # apply interventions
    for var, val in interventions.items():
        if var not in x.index:
            raise ValueError(f"Intervened variable '{var}' not found in base_row.")
        x[var] = val

    # propagate
    for node in topo_order:
        if node in interventions:
            continue  # intervened variables stay fixed

        model_info = scm_models.get(node, None)
        if model_info is None:
            continue  # root or missing model

        parents = model_info["parents"]
        model = model_info["model"]
        parent_vals = np.array([x[p] for p in parents], dtype=float).reshape(1, -1)
        x[node] = float(model.predict(parent_vals)[0])

    return x

Full dataset shape: (19615, 41)
Numeric df shape: (19615, 35)


In [7]:
# ==== BUILD PROMPT FOR QUERY INTERPRETATION ====
def build_schema_text(schema):
    lines = []
    for v in schema:
        lines.append(
            f"- name: {v['name']}\n"
            f"  display_name: {v['display_name']}\n"
            f"  role: {v['role']}\n"
            f"  description: {v['description']}\n"
        )
    return "\n".join(lines)

In [8]:
def make_query_interpret_prompt(user_query, schema):
    def build_schema_text(schema):
        lines = []
        for v in schema:
            lines.append(
                f"- name: {v['name']}\n"
                f"  display_name: {v['display_name']}\n"
                f"  role: {v['role']}\n"
                f"  description: {v['description']}\n"
            )
        return "\n".join(lines)

    schema_text = build_schema_text(schema)

    prompt = f"""You are a causal query interpreter.

User's question:
\"\"\"{user_query}\"\"\"

You are given a list of numeric variables you can intervene on or measure:

{schema_text}

Your job:
1. Decide which variables the user implicitly wants to INTERVENE on (change), if any.
2. Decide which variables should be the TARGET outcomes of interest.
3. For each intervention, specify:
   - "variable": one of the 'name' fields from the list above (use exact name)
   - "direction": "increase", "decrease", or "set"
   - "magnitude": "small", "medium", "large", or "exact"

4. Classify the query_type as one of:
   - "descriptive"      → asking to describe or explain what is happening
   - "predictive"       → asking what will happen without changing anything
   - "interventional"   → asking about the effect of changing a variable in general
   - "counterfactual"   → asking what would have happened in this specific call if something had been different

If the user is only asking for an explanation of what is happening (no change), then:
- query_type should be "descriptive"
- "interventions" should be an empty list []
- "targets" should list the most relevant variables to describe.

Important rules:
- Use ONLY variable names from the provided list for "variable" and in "targets".
- Do NOT invent new variable names.
- If no clear intervention is implied, leave "interventions": [].
- Always output valid JSON.

Output STRICTLY in this JSON format and nothing else:

{{
  "query_type": "descriptive|predictive|interventional|counterfactual",
  "interventions": [
    {{
      "variable": "name_here",
      "direction": "increase|decrease|set",
      "magnitude": "small|medium|large|exact"
    }}
  ],
  "targets": ["var_name_1", "var_name_2", "..."]
}}
"""
    return prompt

In [13]:
prompt = make_query_interpret_prompt(USER_QUERY, schema)


# ==== CALL MISTRAL TO INTERPRET QUERY ====
def call_mistral(prompt, max_new_tokens=512):
    inputs = tokenizer(
        [prompt],
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            num_beams=1,
        )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text

raw_output = call_mistral(prompt)
print("Raw model output:\n", raw_output)

Raw model output:
 You are a causal query interpreter.

User's question:
"""How would the interaction have changed if the agent had apologized at the moment the customer first expressed frustration?"""

You are given a list of numeric variables you can intervene on or measure:

- name: emo_sadness_mean
  display_name: Sadness Mean
  role: customer_emotion
  description: Average intensity of sadness expressed by the customer throughout the call (scaled 0–1).

- name: emo_sadness_median
  display_name: Sadness Median
  role: customer_emotion
  description: Median sadness intensity across customer utterances (scaled 0–1).

- name: emo_sadness_var
  display_name: Sadness Variance
  role: emotion_variability
  description: Variance in sadness intensity across the call.

- name: emo_sadness_max
  display_name: Sadness Max
  role: customer_emotion
  description: Maximum sadness intensity reached during the call (scaled 0–1).

- name: emo_approval_mean
  display_name: Approval Mean
  role: pos

In [15]:
# ==== EXTRACT JSON FROM MODEL OUTPUT ====
import re

import json

def extract_plan_from_output(raw_output: str):
    """
    Extract the single JSON plan object from the model output.
    We:
      1) Take only the part after the last 'Answer:' marker.
      2) Find the first '{' and the last '}' in that segment.
      3) Parse that as JSON.
    """
    segment = raw_output

    # 1. Keep only text after the LAST "Answer:"
    if "Answer:" in segment:
        segment = segment.split("Answer:")[-1]

    # 2. Find JSON boundaries
    start = segment.find("{")
    end = segment.rfind("}")

    if start == -1 or end == -1 or end <= start:
        raise ValueError("Could not find a JSON object in model output.")

    json_str = segment[start:end+1]

    # 3. Clean potential smart quotes (just in case)
    json_str = (json_str.replace("“", '"')
                         .replace("”", '"')
                         .replace("‘", '"')
                         .replace("’", '"'))

    # 4. Parse JSON
    plan = json.loads(json_str)
    return plan

plan = extract_plan_from_output(raw_output)
print("\nParsed plan:\n", plan)


Parsed plan:
 {'query_type': 'counterfactual', 'interventions': [{'variable': 'emo_sadness_mean', 'direction': 'set', 'magnitude': 'exact'}, {'variable': 'emo_anger_mean', 'direction': 'decrease', 'magnitude': 'medium'}], 'targets': ['emo_sadness_mean', 'emo_sadness_median', 'emo_anger_mean', 'emo_anger_median', 'num_customer_turns']}


In [16]:
# ==== MAP MAGNITUDE TO NUMERIC VALUE ====
def magnitude_to_new_value(current_val, var_meta, magnitude, direction, df):
    """
    Convert (direction, magnitude) into a numeric value for intervention.
    Uses variable's scale_min/scale_max and/or dataset stats.
    """
    name = var_meta["name"]
    vmin = var_meta.get("scale_min", float(df[name].min()))
    vmax = var_meta.get("scale_max", float(df[name].max()))
    span = vmax - vmin if vmax > vmin else 1.0

    # If direction is "set", we choose an absolute target value.
    if direction == "set":
        if magnitude == "exact":
            # Use dataset median as a neutral "exact" target
            base = float(df[name].median())
        elif magnitude == "small":
            base = vmin + 0.25 * span
        elif magnitude == "medium":
            base = vmin + 0.50 * span
        elif magnitude == "large":
            base = vmin + 0.75 * span
        else:
            base = float(df[name].mean())
        return float(np.clip(base, vmin, vmax))

    # For increase / decrease, we use a step around the current value
    if magnitude == "small":
        step = 0.15 * span
    elif magnitude == "medium":
        step = 0.30 * span
    elif magnitude == "large":
        step = 0.50 * span
        step = min(step, span)  # clamp
    else:
        step = 0.20 * span

    if direction == "decrease":
        step = -step

    new_val = current_val + step
    return float(np.clip(new_val, vmin, vmax))

In [17]:
# ========================
# RUN SCM FOR SELECTED TRANSCRIPTS
# ========================
results_for_llm = []   # list of dicts, one per transcript

interventions_spec = plan.get("interventions", [])
target_vars = plan.get("targets", [])

for tid in TEST_TRANSCRIPT_IDS:
    # find row for this transcript
    matches = df_full[df_full["transcript_id"] == tid]
    if matches.empty:
        print(f"WARNING: transcript_id {tid} not found in dataset.")
        continue

    base_full = matches.iloc[0]
    base_numeric = base_full[NUMERIC_VARS]

    interventions_numeric = {}

    for inter in interventions_spec:
        var = inter["variable"]
        direction = inter.get("direction", "increase")
        magnitude = inter.get("magnitude", "small")

        if var not in NUMERIC_VARS:
            print(f"WARNING: intervention var {var} not a known numeric variable. Skipping.")
            continue

        var_meta = schema_by_name[var]
        cur_val = float(base_numeric[var])

        new_val = magnitude_to_new_value(cur_val, var_meta, magnitude, direction, df_numeric)
        interventions_numeric[var] = new_val

    if not interventions_numeric:
        print(f"No valid interventions for transcript_id {tid}. Skipping SCM.")
        continue

    # simulate SCM
    cf_numeric = simulate_scm(base_numeric, interventions_numeric, bundle)

    # collect before/after for targets
    target_summaries = []
    for tvar in target_vars:
        if tvar not in NUMERIC_VARS:
            print(f"WARNING: target var {tvar} not numeric. Skipping.")
            continue
        before = float(base_numeric[tvar])
        after = float(cf_numeric[tvar])
        target_summaries.append({
            "variable": tvar,
            "before": before,
            "after": after,
            "delta": after - before,
        })

    results_for_llm.append({
        "transcript_id": tid,
        "query_type": plan.get("query_type", "counterfactual"),
        "interventions": interventions_numeric,
        "targets": target_summaries,
    })

# ========================
# SHOW RESULT SUMMARY
# ========================
print("\n--- SCM RESULTS FOR LLM ---\n")
for item in results_for_llm:
    print(f"Transcript: {item['transcript_id']}")
    print("Interventions (numeric):")
    for k, v in item["interventions"].items():
        print(f"  - {k}: {v}")
    print("Targets (before → after, Δ):")
    for t in item["targets"]:
        print(f"  - {t['variable']}: {t['before']:.4f} → {t['after']:.4f} (Δ = {t['delta']:.4f})")
    print()

# Save for downstream LLM
OUTPUT_JSON_PATH = "/kaggle/working/scm_results_for_llm.json"
with open(OUTPUT_JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(results_for_llm, f, indent=4, ensure_ascii=False)

print("Saved SCM results for LLM to:", OUTPUT_JSON_PATH)


--- SCM RESULTS FOR LLM ---

Transcript: 1c99b47c-4e80-49d5-b3c6-52b0c3230fa8
Interventions (numeric):
  - emo_sadness_mean: 0.0106707176353127
  - emo_anger_mean: 0.0008982101895857
Targets (before → after, Δ):
  - emo_sadness_mean: 0.0075 → 0.0107 (Δ = 0.0032)
  - emo_sadness_median: 0.0020 → 0.0015 (Δ = -0.0005)
  - emo_anger_mean: 0.0104 → 0.0009 (Δ = -0.0095)
  - emo_anger_median: 0.0025 → 0.0013 (Δ = -0.0012)
  - num_customer_turns: 14.0000 → 17.0033 (Δ = 3.0033)

Saved SCM results for LLM to: /kaggle/working/scm_results_for_llm.json
